In [265]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest

In [338]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('train.csv', index_col='Id') 

print(X.shape)
X_test = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

print(y.shape)
numerical_cols = [col for col in X.columns if X[col].dtype != "object"]


object_cols = [col for col in X.columns if X[col].dtype == "object"]

X

(1460, 80)
(1460,)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml


In [215]:
X_test.shape

(1459, 79)

### 1. Take Care of Missing Values

In [317]:
## In this case, I will replace missing values with Median of corresponding values
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


scalar =StandardScaler()

in_median = SimpleImputer(strategy = 'median')


X_numerical = X[numerical_cols]
X_test_numerical = X_test[numerical_cols]


imputed_X = pd.DataFrame(in_median.fit_transform(X_numerical))
imputed_X_test = pd.DataFrame(in_median.transform(X_test_numerical))



imputed_X.columns = X[numerical_cols].columns
imputed_X_test.columns = X_test[numerical_cols].columns

print('The size of imputedX ' , imputed_X.shape)
#imputed_X = pd.DataFrame(in_median.fit_transform()



The size of imputedX  (1460, 36)


In [362]:
# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X[col]) == set(X_test[col])]


low_cardinality_cols = [col for col in good_label_cols if X[col].nunique() < 10]


nan_columns = [col for col in low_cardinality_cols if X[col].isna().any().sum() > 0]


good_cols = list(set(low_cardinality_cols)-set(nan_columns))


    
print('Shape of X', X.shape)

#y = y[good_cols]

X_object_columns = X[good_cols]
X_test_object_columns = X_test[good_cols]



print(X_object_columns.shape)


X_object_columns.columns = X[good_cols].columns
X_test_object_columns.columns = X_test[good_cols].columns


oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse = False)


OH_cols_X = pd.DataFrame(oh_encoder.fit_transform(X_object_columns))
OH_cols_Xtest = pd.DataFrame(oh_encoder.transform(X_test_object_columns))




final_X = pd.concat([imputed_X, OH_cols_X], axis=1)
final_X_test = pd.concat([imputed_X_test, OH_cols_Xtest], axis=1)


Shape of X (1460, 79)
(1460, 15)


In [363]:
final_X = final_X.iloc[0:1460,:]

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(final_X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [364]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, X_valid)

ValueError: Number of labels=292 does not match number of samples=1168

### Getting the columns with categorical variable

In [84]:
s = (home_data.dtypes == 'object')
categorical_var = list(s[s].index)

In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Dropping the columns with categorical data

In [85]:
drop_X_train = X_train.select_dtypes(exclude= ['object'])
drop_X_valid = X_valid.select_dtypes(exclude= ['object'])

### Checking for the NaN values

In [80]:
missing_values_byColumn = drop_X_train.isnull().sum()
missing_values_byColumn[missing_values_byColumn>0]

##Getting the name of the columns that includes a missing value

col_name = [col for col in drop_X_train.columns if drop_X_train[col].isnull().any()]
col_name

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

### Checking the effectiveness of the result

In [86]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


### Normalization

In [65]:
from sklearn.preprocessing import StandardScaler

#features = ['LotArea','MiscVal','2ndFlrSF','PoolArea','BsmtFinSF2','GrLivArea','TotalBsmtSF', 'GarageCars']

X = home_data[features]
X.isna().sum()|

SyntaxError: invalid syntax (<ipython-input-65-e3bc7a0cd50b>, line 5)

In [35]:
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [37]:
#Getting the best max_leaf_nodes
def get_accuracy(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    acc = MAE(val_y, preds_val)
    
    return acc

    
max_leaf = [x for x in range(50, 250)]

 
best_accuracy = {}  
for i in range(len(max_leaf)):
    best_accuracy[i] = get_accuracy(max_leaf[i], train_X,val_X, train_y, val_y)

In [39]:
#Getting best n_estimators
def get_accuracy(n_estimators, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=4, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    acc = MAE(val_y, preds_val)
    return acc

best_estimator = [x for x in range(50, 250)]
 
best_accuracy_n_estimator = {}  
for i in range(len(max_leaf)):
    best_accuracy_n_estimator[i] = get_accuracy(best_estimator[i], train_X,val_X, train_y, val_y)

In [38]:
#Best max_leaf
min_key = min(best_accuracy, key=best_accuracy.get)
print(min_key)

4


In [40]:
#Best n_estimator
min_key = min(best_accuracy_n_estimator, key=best_accuracy_n_estimator.get)
print(min_key)

68


In [41]:
rf_model_on_full_data = RandomForestRegressor(n_estimators=68, max_leaf_nodes=4)
rf_model_on_full_data.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=4,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=68, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [50]:
test_data = pd.read_csv("test.csv")
test_X = test_data[features]
print(test_X.isna().sum())
test_X.fillna(0, inplace=True)


sc = StandardScaler()
sc.fit(test_X)
test_X = sc.transform(test_X)

#dummies = pd.get_dummies(test_X.Neighborhood)
#df_dummies = pd.concat([test_X, dummies], axis = 'columns')
#df_dummies.drop(['Neighborhood'], axis='columns', inplace=True)
#test_X = df_dummies


test_preds = rf_model_on_full_data.predict(test_X)
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission3.csv', index=False)




LotArea        0
MiscVal        0
2ndFlrSF       0
PoolArea       0
BsmtFinSF2     1
GrLivArea      0
TotalBsmtSF    1
GarageCars     1
dtype: int64


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
